In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qml.kernels import gaussian_kernel
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from qml.math import cho_solve
from sklearn.model_selection import train_test_split
from qml.math import cho_solve

In [2]:
np.random.seed(27)

def f1(x):
    return np.sin(x[:, 0]) + np.cos(x[:, 1]) + np.exp(x[:, 2]) + np.sin(x[:, 3]) + np.cos(x[:, 4])
def f2(x):
    return np.cos(x[:, 0]) + np.exp(x[:, 1]) + np.sin(x[:, 2]) + np.cos(x[:, 3]) + np.exp(x[:, 4])
def f3(x):
    return np.sin(x[:, 0]) + np.exp(x[:, 1]) + np.cos(x[:, 2]) + np.sin(x[:, 3]) + np.exp(x[:, 4])
# Generate 3000 samples of 5 dimensional input X with random numbers between 0 and 1
X = np.random.rand(3000, 5)

# Generate 3 dimensional output Y using output functions
Y = np.zeros((3000, 3))
Y[:, 0] = f1(X)
Y[:, 1] = f2(X)
Y[:, 2] = f3(X)




In [3]:
def krr_qml(x_train,x_test,y_train,y_test,sigma):
    K=gaussian_kernel(x_train,x_train,sigma=sigma)
    K[np.diag_indices_from(K)] += 1e-9
    alpha = cho_solve(K, y_train)
    Ks=gaussian_kernel(x_train,x_test,sigma)
    y_predict=np.dot(alpha,Ks) 
    return np.mean(np.abs(y_predict - y_test))


In [4]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, Y, shuffle=False, train_size=0.8)

sigmas = range(0, 5001, 100)

def hypertuning(sigmas):
    """
    Hypertuning function which finds the optimal kernel width for a given dataset. The function minimises the MAE of KRR.

    Parameters:
        sigmas(range): The kernel widths to try. 
        
    Returns:
        optimal_sigma(int): The kernel width with the lowest MAE.
    """
    min_mae = np.inf
    optimal_sigma = None
    for sigma_val in sigmas:
        mae_templ = krr_qml(X_train1[0:200], X_test1[0:50], Y_train[0:200], Y_test[0:50], sigma_val)
        if mae_templ < min_mae:
            min_mae = mae_templ
            optimal_sigma = sigma_val
    return optimal_sigma     

optimal_sigmas = []
for i in range(3):
    Y_train = Y_train1[:,i]
    Y_test = Y_test1[:,i]
    optimal_sigma = hypertuning(sigmas)
    optimal_sigmas.append(optimal_sigma)
    print(f"Optimal Sigma value for component {i+1}: {optimal_sigma}")

Optimal Sigma value for component 1: 100
Optimal Sigma value for component 2: 100
Optimal Sigma value for component 3: 100
